In [4]:

# import yaml
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

from biotorch.benchmark.run import Benchmark
from biotorch.module.biomodule import BioModule

In [6]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [7]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [ ]:
print(train_loader)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [25]:
network = Net()
# network = BioModule(network, mode='fa')
print(network)
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [26]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [27]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      # torch.save(network.state_dict(), '/results/model.pth')
      # torch.save(optimizer.state_dict(), '/results/optimizer.pth')

In [28]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [29]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

C:\Users\patel\AppData\Local\Temp\ipykernel_29740\161431047.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 2.2953, Accuracy: 841/10000 (8%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.286329
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.269986
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.265898
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.297638
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.244451
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.259844
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.188649
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.192837
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.154733
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.910791
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.878963
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.910301
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.669419
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.535789
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.568959
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.511087
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.100953
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.242976
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.011507
Train Epoch: 1 [12160/6

In [ ]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
#         self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
#         self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=2)
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
#         self.relu3 = nn.ReLU()
#         self.fc = nn.Linear(in_features=128, out_features=10)

#     def forward(self, x):
#         out = self.pool1(self.relu1(self.conv1(x)))
#         out = self.pool2(self.relu2(self.conv2(out)))
#         out = self.relu3(self.conv3(out))
#         out = F.avg_pool2d(out, out.size()[3])
#         out = out.view(out.size(0), -1)
#         return self.fc(out)